# `agg`, `filter`, `transform`, and `apply` GroupBy Methods

## EECS 398-003: Practical Data Science, Fall 2024

### Live from Office Hours

In office hours on Tuesday, October 8th, we walked through examples of how `agg`, `filter`, `transform`, and `apply` work. Watch a walkthrough of the notebook here: https://www.loom.com/share/4171bfc0e71c4722b10a6c8472d08cb5?sid=2dde2a35-e2a1-4619-8a16-5efb8570362d.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

In [ ]:
penguins = sns.load_dataset('penguins').dropna()
penguins

## agg

---

Average bill length per species.

In [ ]:
penguins.groupby('species')['bill_length_mm'].mean()

Average bill length and average bill depth per species.

In [ ]:
penguins.groupby('species')[['bill_length_mm', 'bill_depth_mm']].mean()

In [ ]:
# penguins.groupby('species').mean() # Need to select relevant columns first!

In [ ]:
penguins.groupby('species')['bill_length_mm'].agg(['min', 'max'])

In [ ]:
penguins.groupby('species').agg({'bill_length_mm': ['max', 'min'],
                                 'bill_depth_mm': 'min',
                                 'island': 'nunique'})

In [ ]:
penguins.groupby('species')['bill_length_mm'].mean()

If giving `agg` a function, the function should:
- Take in a Series.
- Return a single value.

In [ ]:
(
    penguins
    .groupby('species')
    ['bill_length_mm']
    .agg(lambda s: np.percentile(s, 75) - np.percentile(s, 25))
)

## filter

---

- Querying: Keeping **rows** that satisfy conditions.
- Filtering: Keeping **groups** that satisfy conditions.

Show me the penguins that have a bill length over 40 mm.

In [ ]:
penguins[penguins['bill_length_mm'] > 40]

Show me the **species** with an average bill length over 40 mm.

In [ ]:
penguins

`filter` takes in a function, that:
- Takes in a DataFrame.
- Returns a BOOLEAN.

In [ ]:
penguins.groupby('species').filter(lambda df: df['bill_length_mm'].mean() > 40)

In [ ]:
mean_lengths = penguins.groupby('species')['bill_length_mm'].mean()

In [ ]:
mean_lengths

In [ ]:
mean_lengths[mean_lengths > 40].index

In [ ]:
penguins[penguins['species'].isin(mean_lengths[mean_lengths > 40].index)]

Here, the function that `filter` accepts:
- Takes in a **Series** (which we've still called `df`).
- Returns a BOOLEAN.

In [ ]:
penguins.groupby('species')['bill_length_mm'].filter(lambda df: df.mean() > 40)

## transform

---

In [ ]:
penguins

In [ ]:
penguins = sns.load_dataset('penguins')
penguins

In [ ]:
penguins.isna().sum()

$$\text{z-score} = \frac{\text{value} - \text{mean of column}}{\text{SD of column}}$$

In [ ]:
def z_score(col):
    return (col - col.mean()) / col.std()

In [ ]:
z_score(penguins['bill_length_mm'])

$z$-scoring the bill lengths, **separately** for each species. That is, when doing the transformation for a particular species, it only uses the mean and SD for that species, not all penguins overall.

In [ ]:
penguins.groupby('species')['bill_length_mm'].transform(z_score)

`transform` takes in a function, that:
- Takes in a Series.
- Returns a Series.

## apply

---

In [ ]:
penguins.groupby('species').apply(lambda df: df['bill_length_mm'].max())

equivalent to:

In [ ]:
penguins.groupby('species')['bill_length_mm'].max()

The bill length of the second heaviest penguin per species.

In [ ]:
def bill_length_of_second_heaviest(df):
    return df.sort_values('body_mass_g', ascending=False).iloc[1].loc['bill_length_mm']

In [ ]:
bill_length_of_second_heaviest(penguins)

In this case, `apply` is taking in a function, that:
- Takes in a DataFrame.
- Returns a number. (but it could also return something else!)

In [ ]:
penguins.groupby('species').apply(bill_length_of_second_heaviest)

Find me all the rows for the three heaviest penguins per species.

In [ ]:
def three_heaviest_penguins(penguins):
    return penguins.sort_values('body_mass_g', ascending=False).head(3)

In [ ]:
three_heaviest_penguins(penguins)

In this case, `apply` takes in a function that:
- Takes in a DataFrame.
- Returns a DataFrame.

In [ ]:
penguins.groupby('species').apply(three_heaviest_penguins)

Moral of the story: `apply` is more general purpose than `agg`/`transform` (I believe `filter` is an edge-case).

In [ ]:
penguins

In [ ]:
penguins['species'].str[0]

In [ ]:
penguins.assign(first_letter_of_species = penguins['species'].str[0])